# Utils

In [18]:
import numpy as np
import numpy.linalg as linalg
from typing import Tuple, List, Callable
from math import sqrt
import itertools
from enum import Enum
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from timeit import default_timer as timer

In [19]:
def get_circle_radius(matrix, row_index, is_squared = False):
    if is_squared:
        return (matrix[row_index] ** 2).sum() - (matrix[row_index][row_index] ** 2)

    return abs(matrix[row_index]).sum() - abs(matrix[row_index][row_index])

In [42]:
def get_gershgorin_circles(matrix):
    circles = []
    for i in range(matrix.shape[0]):
        radius = get_circle_radius(matrix, i)
        center = matrix[i][i]
        circles.append((center - radius, center + radius))
    return sorted(circles, key=lambda p: p[0])

In [21]:
def rotate(matrix, index):
    i, j = index

    matrix_ii, matrix_ij, matrix_jj = matrix[i][i], matrix[i][j], matrix[j][j]

    x = -2 * matrix_ij
    y = matrix_ii - matrix_jj

    if y == 0:
        cos_phi = sqrt(2) / 2
        sin_phi = sqrt(2) / 2
    else:
        cos_phi = sqrt(1 / 2 * (1 + abs(y) / sqrt(x ** 2 + y ** 2)))
        sin_phi = np.sign(x * y) * abs(x) / (2 * cos_phi * sqrt(x ** 2 + y ** 2))

    matrix[i][i] = cos_phi ** 2 * matrix_ii - 2 * sin_phi * cos_phi * matrix_ij + sin_phi ** 2 * matrix_jj
    matrix[j][j] = sin_phi ** 2 * matrix_ii + 2 * sin_phi * cos_phi * matrix_ij + cos_phi ** 2 * matrix_jj

    matrix[i][j] = (cos_phi ** 2 - sin_phi ** 2) * matrix_ij + sin_phi * cos_phi * (matrix_ii - matrix_jj)
    matrix[j][i] = (cos_phi ** 2 - sin_phi ** 2) * matrix_ij + sin_phi * cos_phi * (matrix_ii - matrix_jj)

    for k in range(matrix.shape[0]):
        if k != i and k != j:
            matrix_ik, matrix_jk = matrix[i][k], matrix[j][k]

            matrix[i][k] = cos_phi * matrix_ik - sin_phi * matrix_jk
            matrix[k][i] = cos_phi * matrix_ik - sin_phi * matrix_jk

            matrix[j][k] = sin_phi * matrix_ik + cos_phi * matrix_jk
            matrix[k][j] = sin_phi * matrix_ik + cos_phi * matrix_jk


In [22]:
def find_abs_max_index(matrix):
    matrix = matrix - np.diag(np.diag(matrix))
    return np.unravel_index(np.argmax(abs(matrix)), matrix.shape)

In [23]:
def get_line_by_line_indices(shape):
    indicies = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            if i != j:
                indicies.append((i, j))
    return itertools.cycle(indicies)

In [24]:
def get_vector_of_square_sums(matrix):
    return np.array([get_circle_radius(matrix, i, is_squared=True) for i in range(matrix.shape[0])])


def get_pivot_with_max_radius(matrix, vector):
    i = vector.argmax()
        
    matrix = matrix - np.diag(np.diag(matrix))
    j = abs(matrix[i]).argmax()

    return i, j

In [25]:
class PivotStrategy(Enum):
    ABS_MAX = 'abs_max'
    LINE_BY_LINE = 'line_by_line'
    MAX_RADIUS = 'max_radius'

In [46]:
def find_eigenvalues(
        matrix,
        *,
        eps,
        strategy: PivotStrategy,
        limit = 10000
):
    matrix = np.copy(matrix)

    circle_indicies = get_line_by_line_indices(matrix.shape)
    square_sums_vector = get_vector_of_square_sums(matrix)

    for step in range(limit):
        if strategy is PivotStrategy.MAX_RADIUS:
            if all(get_circle_radius(matrix, i, is_squared=True) < eps ** 2 for i in range(matrix.shape[0])):
                return np.diag(matrix), step
        else:
            if all(get_circle_radius(matrix, i) < eps for i in range(matrix.shape[0])):
                return np.diag(matrix), step

        if strategy is PivotStrategy.MAX_RADIUS:
            index = get_pivot_with_max_radius(matrix, square_sums_vector)
            square_sums_vector[index[0]] = get_circle_radius(matrix, index[0], is_squared=True)
            square_sums_vector[index[1]] = get_circle_radius(matrix, index[1], is_squared=True)
            rotate(matrix, index)
        elif strategy is PivotStrategy.ABS_MAX:
            index = find_abs_max_index(matrix)
            rotate(matrix, index)

    return np.diag(matrix), step + 1

In [27]:
def run_benchmark_matrix(matrix, eps):
    true_values, _ = linalg.eig(matrix)

    abs_max_values, abs_max_steps = find_eigenvalues(matrix, eps=eps, strategy=PivotStrategy.ABS_MAX)
    radius_values, radius_steps = find_eigenvalues(matrix, eps=eps, strategy=PivotStrategy.MAX_RADIUS)

    return pd.Series([
        abs_max_steps,
        linalg.norm(abs(np.sort(true_values)) - abs(np.sort(abs_max_values))),
        radius_steps,
        linalg.norm(abs(np.sort(true_values)) - abs(np.sort(radius_values)))
    ])

In [31]:
def compare_eps_and_steps(data, n):
    data = data[data.n == n]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.abs_max_steps, name='Максимальный по модулю')
    fig.add_scatter(x=data.eps, y=data.radius_steps, name='Круг максимального радиуса')

    fig.update_xaxes(title='ε', type='log', tickformat='.0e', autorange='reversed')
    fig.update_yaxes(title='Количество шагов')

    fig.update_layout(title=f'Зависимость количества шагов от ε (n = {n})', legend_title_text='Стратегия')

    fig.show()

Матрица Гильберта

In [34]:
def generate_matrix(element_factory, size):
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )


def get_hilbert_matrix(size):
    return generate_matrix(lambda row, column: 1 / (row + column - 1), size)

In [39]:
hilbert_data = pd.DataFrame(product(range(2, 11), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
hilbert_data[
    [
        'abs_max_steps',
        'abs_max_error', 
        'radius_steps',
        'radius_error',
    ]
] = hilbert_data.apply(
    func=lambda row: run_benchmark_matrix(get_hilbert_matrix(int(row.n)), row.eps),
    axis=1,
)
hilbert_data

,n,eps,abs_max_steps,abs_max_error,radius_steps,radius_error
0,2,1.000000e-02,1.0,2.237726e-16,1.0,2.237726e-16
1,2,1.000000e-03,1.0,2.237726e-16,1.0,2.237726e-16
2,2,1.000000e-04,1.0,2.237726e-16,1.0,2.237726e-16
3,2,1.000000e-05,1.0,2.237726e-16,1.0,2.237726e-16
4,2,1.000000e-06,1.0,2.237726e-16,1.0,2.237726e-16
...,...,...,...,...,...,...
121,10,1.000000e-11,144.0,7.367068e-15,150.0,1.155914e-15
122,10,1.000000e-12,154.0,1.780342e-15,160.0,6.297124e-16
123,10,1.000000e-13,161.0,1.780342e-15,174.0,6.297137e-16
124,10,1.000000e-14,169.0,1.780342e-15,175.0,6.297137e-16


In [40]:
compare_eps_and_steps(hilbert_data, 3)
compare_eps_and_steps(hilbert_data, 5)
compare_eps_and_steps(hilbert_data, 10)

In [49]:
matrix = get_hilbert_matrix(5)
print(get_gershgorin_circles(matrix))
print(sorted(find_eigenvalues(matrix, eps=1e-12, strategy=PivotStrategy.ABS_MAX)[0]))
print(sorted(find_eigenvalues(matrix, eps=1e-12, strategy=PivotStrategy.MAX_RADIUS)[0]))

[(-0.7833333333333334, 1.45), (-0.6928571428571428, 1.0928571428571427), (-0.5988095238095239, 0.8845238095238095), (-0.5234126984126983, 0.7456349206349207), (-0.2833333333333332, 2.283333333333333)]
[3.2879287721777533e-06, 0.0003058980401511705, 0.011407491623419811, 0.20853421861101334, 1.5670506910982311]
[3.2879287721775013e-06, 0.0003058980401511658, 0.011407491623419794, 0.2085342186110133, 1.567050691098231]


Пакулина

In [57]:
A = np.array([
    [15., 3., 5., 2.], 
    [-5., -23., 2., -1.], 
    [12., -9., 35., -1.], 
    [9., 2., 6., 36.]
])

print(f'Точные значения: {[-22.090929928620027940, 11.170008850837157393, 38.387070273036965682, 35.533850804745904865]}')
print(f'Круги Гершгорина: {sorted(get_gershgorin_circles(A), key=lambda x: x[0])}')

Точные значения: [-22.090929928620028, 11.170008850837158, 38.38707027303697, 35.5338508047459]
Круги Гершгорина: [(-31.0, -15.0), (5.0, 25.0), (13.0, 57.0), (19.0, 53.0)]


In [60]:
abs_max_result = find_eigenvalues(A, eps=1e-10, strategy=PivotStrategy.ABS_MAX)
np.sort(abs_max_result[0]), abs_max_result[1]

(array([-24.99657431,  10.43567146,  36.189968  ,  41.37093484]), 15)

In [61]:
max_radius_result = find_eigenvalues(A, eps=1e-10, strategy=PivotStrategy.MAX_RADIUS)
np.sort(max_radius_result[0]), max_radius_result[1]

(array([-24.99657431,  10.43567146,  36.189968  ,  41.37093484]), 12)